#### Нейросетевой классификатор на базе предобученной модели rubert-tiny2

In [ ]:
!pip install transformers

In [1]:
import re
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
from utils import *

In [ ]:
from google.colab import drive

In [5]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [6]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [7]:
from transformers import logging, get_cosine_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [8]:
def set_seed(seed):
    np.random.seed(seed)
    torch.random.manual_seed(seed)
    torch.cuda.random.manual_seed(seed)
    torch.cuda.random.manual_seed_all(seed)

def get_available_device():
    cuda = torch.cuda.is_available()
    return torch.device('cuda' if cuda else 'cpu')

In [9]:
def clean(s):
    s = re.sub('\n', ' ', s)
    s = re.sub('<\?xml.+\?>', ' ', s)
    s = re.sub('<\/?[A-Za-z]*>', ' ', s)
    s = re.sub('[0-9]+(\.|,)?[0-9]*', ' num ', s)
    s = re.sub('([a-zа-яёй])([A-ZА-ЯЁЙ])', '\g<1> \g<2>', s)
    s = re.sub('[^A-Za-zА-ЯЁЙа-яёй0-9.,;:!?()"\s\-\']', ' ', s)
    return re.sub('\s{2,}', ' ', s).strip()

* *отличия: не удаляем символы `.,;:!?()"`, не приводим к нижнему регистру*

In [10]:
def tokenize(titles, texts, tokenizer):
    
    params = {'max_length': 512, 
              'truncation': True, 
              'padding': 'max_length', 
              'add_special_tokens': True}
    
    input_ids = torch.zeros((len(titles), 512), dtype=torch.int32)
    
    for i in tqdm(range(len(titles))):
        title, text = clean(titles[i]), clean(texts[i])
        ids = tokenizer.encode(title, text, **params)
        input_ids[i] = torch.tensor(ids, dtype=torch.int32)
    
    return input_ids

* *самая простая (и не самая эффективная в плане работы загрузчика) токенизация*
* *если количество получаемых токенов больше 512, то обрезаем последовательность*
* *если количество получаемых токенов меньше 512, то дополняем последовательность (pad_token)*
* *токенизируем сразу и title и text (используем токен sep): `cls title sep text sep`*
* *на выходе сразу получаем тензор индексов токенов размерности [кол-во документов, 512]*

In [11]:
def load_model(name):
    
    params = {
        'num_labels': 6, 
        'output_attentions': False, 
        'output_hidden_states': True
    }
    
    tokenizer = AutoTokenizer.from_pretrained(name)
    model = AutoModelForSequenceClassification.from_pretrained(name, **params)
    return model, tokenizer

* *функция для загрузки предобученной модели и токенайзера*

In [ ]:
drive.mount('/content/drive')

In [11]:
!unzip /content/drive/MyDrive/shared/topic/data.zip -d data

Archive:  /content/drive/MyDrive/shared/topic/raw.zip
  inflating: data/raw.csv            


* *загружаем и распаковываем данные (сейчас мы на google colab - будем обучаться на gpu)*

In [12]:
set_seed(42)
logging.set_verbosity_error()
model, tokenizer = load_model('cointegrated/rubert-tiny2')

* *загружаем предобученную модель и токенайзер*

In [13]:
tokenizer

PreTrainedTokenizerFast(name_or_path='cointegrated/rubert-tiny2', vocab_size=83828, model_max_len=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

* *размер словаря: 83828 токенов (не сможем использовать uint16)*
* *максимальная длина входной последовательности: 2048 (вместо 512, видимо, нестандартный attention)*

In [22]:
model.bert.encoder.layer[-1].output

BertOutput(
  (dense): Linear(in_features=600, out_features=312, bias=True)
  (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

* *размерность hidden_state (размер эмбеддингов) = 312*

In [12]:
df = pd.read_csv('data/raw.csv') #.sample(100)
labels = df['topic'].map(load_json('data/labels.json'))
labels = torch.tensor(labels.values, dtype=torch.int8)
input_ids = tokenize(df['title'].values, df['text'].values, tokenizer)
torch.save(input_ids, 'data/input_ids.pt')
torch.save(labels, 'data/labels.pt')

  0%|          | 0/432158 [00:00<?, ?it/s]

* *загружаем датафрейм, кодируем целевые метки*
* *токенизируем все документы, сохраняем тензоры*

In [13]:
input_ids.shape, labels.shape

(torch.Size([432158, 512]), torch.Size([432158]))

* *432158 документов, по 512 индексов токенов на каждый*

In [14]:
dataset = TensorDataset(input_ids, labels)
train_size = int(0.800 * len(dataset))
valid_size = len(dataset) - train_size

In [15]:
generator = torch.Generator().manual_seed(42)
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size], generator)
train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=32)
valid_loader = DataLoader(valid_dataset, sampler=SequentialSampler(valid_dataset), batch_size=32)

In [16]:
len(train_dataset), len(valid_dataset)

(345726, 86432)

* *создаем датасет, разделяем данные, создаем загрузчики*

In [17]:
def metrics_stat(y_true, y_pred):
    return {'acc': accuracy_score(y_true, y_pred),
            'f1': f1_score(y_true, y_pred, average='macro')}

In [ ]:
def nn_epoch(model, optimizer, scheduler, loader, 
             device, pad_token_id=0, train=False):
    
    model = model.to(device)
    torch.set_grad_enabled(train)
    (model.eval, model.train)[int(train)]()
    
    loss_history = []
    pbar, total_loss = tqdm(loader), 0
    preds = torch.empty(0, dtype=torch.int8)
    targets = torch.empty(0, dtype=torch.int8)
  
    for input_ids, labels in pbar:
        
        input_ids = input_ids.to(device)
        labels = labels.to(device).long()
        attn_mask = (input_ids != pad_token_id)
        
        params = {
            'output_attentions': False, 
            'output_hidden_states': False, 
            'attention_mask': attn_mask, 
            'input_ids': input_ids, 
            'labels': labels
        }
        
        if train:
            optimizer.zero_grad()

        output = model(**params)
        logits = output['logits']
        loss = output['loss']
        
        if train: 
            loss.backward()
            optimizer.step()
            scheduler.step()
        
        targets = torch.cat((targets, labels.cpu()))
        preds = torch.cat((preds, torch.argmax(logits.cpu(), 1)))
        pbar.set_description(f'loss: {loss.item():.4}')
        loss_history.append(loss.item())
        total_loss += loss.item()
    
    avg_loss = total_loss / len(loader)
    pbar.set_description(f'loss: {avg_loss:.4}')
    
    return {'preds': preds, 
            'targets': targets, 
            'loss_history': loss_history}

* *универсальная функция: 1 эпоха обучения / валидации, формирование предсказаний*

In [19]:
num_epochs = 3

save_path = '/content/drive/MyDrive/shared/topic/'

device = get_available_device()
pad_token_id = tokenizer.pad_token_id
total_steps = len(train_loader) * num_epochs
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, eps=1e-8)
scheduler = get_cosine_schedule_with_warmup(optimizer, 0, total_steps)

for epoch in range(num_epochs):
    
    res = nn_epoch(
        model, optimizer, scheduler, 
        train_loader, device, pad_token_id, train=True
    )

    with open(f'{save_path}/train_loss_{epoch+1:02}.npy', 'wb') as fd:
        np.save(fd, np.array(res['loss_history'], dtype=np.float32))
    
    print('train', metrics_stat(res['targets'], res['preds']))
    
    res = nn_epoch(
        model, optimizer, scheduler, 
        valid_loader, device, pad_token_id, train=False
    )
    
    with open(f'{save_path}/valid_loss_{epoch+1:02}.npy', 'wb') as fd:
        np.save(fd, np.array(res['loss_history'], dtype=np.float32))
    
    print('valid', metrics_stat(res['targets'], res['preds']))
    torch.save(model.state_dict(), f'{save_path}/model_{epoch+1:02}.pt')

  0%|          | 0/10804 [00:00<?, ?it/s]

train {'acc': 0.9325448476539224, 'f1': 0.9260189201425089}


  0%|          | 0/2701 [00:00<?, ?it/s]

valid {'acc': 0.9442336171788227, 'f1': 0.9391652510413211}


  0%|          | 0/10804 [00:00<?, ?it/s]

...

valid {'acc': 0.9496598482043688, 'f1': 0.945407698642417}


  0%|          | 0/10804 [00:00<?, ?it/s]

train {'acc': 0.980131664960113, 'f1': 0.9781621563825519}


  0%|          | 0/2701 [00:00<?, ?it/s]

valid {'acc': 0.9489193817104776, 'f1': 0.9445447454298083}


* *обучаем модель, сохраняем веса на каждой эпохе*

In [26]:
path = f'{save_path}/model_02.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

* *на 3ей эпохе началось переобучение, нужно подбирать параметры*
* *загружаем сохраненные веса (лучшая модель - после 2ой эпохи)*

In [27]:
res = nn_epoch(model, None, None, valid_loader, device, pad_token_id, train=False)

  0%|          | 0/2701 [00:00<?, ?it/s]

* *формируем предсказания загруженной моделью на валидационной части данных*

In [28]:
accuracy_score(res['targets'], res['preds'])

0.9496598482043688

In [29]:
f1_score(res['targets'], res['preds'], average='macro')

0.945407698642417

In [30]:
print(confusion_matrix(res['targets'], res['preds']))

[[ 8151   191   283   231    87   119]
 [  125 10337   126    76    23    23]
 [  377   263 25731   395    91   396]
 [  261    72   327  9916     9   104]
 [   21     8    32     3 12779    10]
 [  267    46   234   112    39 15167]]


In [31]:
print(classification_report(res['targets'], res['preds']))

              precision    recall  f1-score   support

           0       0.89      0.90      0.89      9062
           1       0.95      0.97      0.96     10710
           2       0.96      0.94      0.95     27253
           3       0.92      0.93      0.93     10689
           4       0.98      0.99      0.99     12853
           5       0.96      0.96      0.96     15865

    accuracy                           0.95     86432
   macro avg       0.94      0.95      0.95     86432
weighted avg       0.95      0.95      0.95     86432



In [32]:
list(load_json('data/labels.json').keys())

['Интернет и СМИ', 'Культура', 'Мир', 'Наука и техника', 'Спорт', 'Экономика']

* *сравнивать с предыдущими моделями не совсем честно, т.к. здесь другой сплит данных*
* *нужно еще раз произвести валидацию этой модели с тем же сплитом, который использовался ранее*
* *и тем не менее, судя по всему, эта модель будет лучшей в плане качества*
* *веса модели занимают ~100Мб дискового пространства*